# Wikipedia Notable Life Expectancies
# [Notebook  : Data Cleaning Part 6](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean6_thanak_2022_07_26.ipynb)
### Context

The
### Objective

The
### Data Dictionary
- Feature: Description

### Importing Libraries

In [1]:
# To structure code automatically
%load_ext nb_black

# To import/export sqlite databases
import sqlite3 as sql

# To save/open python objects in pickle file
import pickle

# To help with reading, cleaning, and manipulating data
import pandas as pd
import numpy as np
import re

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
# To define the maximum number of rows to be displayed in a dataframe
pd.set_option("display.max_rows", 200)

# To supress warnings
# import warnings

# warnings.filterwarnings("ignore")

# To set some visualization attributes
pd.set_option("max_colwidth", 150)

# To play auditory cue when cell has executed, has warning, or has error and set chime theme
import chime

chime.theme("zelda")

<IPython.core.display.Javascript object>

## Data Overview

### [Reading](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_clean5.db), Sampling, and Checking Data Shape

In [2]:
# Reading the dataset
conn = sql.connect("wp_life_expect_clean5.db")
data = pd.read_sql("SELECT * FROM wp_life_expect_clean5", conn)

# Making a working copy
df = data.copy()

# Checking the shape
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# Checking first 2 rows of the data
df.head(2)

There are 98041 rows and 44 columns.


,day,name,info,link,num_references,year,month,info_parenth,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,info_3_0,info_3_1,info_3_2,info_4_0,info_4_1,info_4_2,info_5_0,info_5_1,info_5_2,info_6_0,info_6_1,info_7_0,info_8_0,info_8_1,info_9_0,info_10_0,info_11_0,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
0,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21,1994,January,None,86.0,None,United Kingdom of Great Britain and Northern Ireland,None,None,3.091042,ballet designer,director,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,1,0,0,0,0,0,0,1
1,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12,1994,January,None,68.0,None,Ireland,None,None,2.564949,writer,None,None,and academic,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,0,0,0,1,0,0,0,1


<IPython.core.display.Javascript object>

In [3]:
# Checking last 2 rows of the data
df.tail(2)

,day,name,info,link,num_references,year,month,info_parenth,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,info_3_0,info_3_1,info_3_2,info_4_0,info_4_1,info_4_2,info_5_0,info_5_1,info_5_2,info_6_0,info_6_1,info_7_0,info_8_0,info_8_1,info_9_0,info_10_0,info_11_0,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
98039,9,Aamir Liaquat Hussain,", 50, Pakistani journalist and politician, MNA .",https://en.wikipedia.org/wiki/Aamir_Liaquat_Hussain,99,2022,June,", since",50.0,None,Pakistan,None,"2002 2007, since 2018",4.605170,MNA,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,1,0,0,1,0,0,0,2
98040,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3,2022,June,None,86.0,None,"China, People's Republic of",None,None,1.386294,member of the Academy of Engineering,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,0,0,0,0,0,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

In [4]:
# Checking a sample of the data
df.sample(5)

,day,name,info,link,num_references,year,month,info_parenth,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,info_3_0,info_3_1,info_3_2,info_4_0,info_4_1,info_4_2,info_5_0,info_5_1,info_5_2,info_6_0,info_6_1,info_7_0,info_8_0,info_8_1,info_9_0,info_10_0,info_11_0,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
34675,22,Bob Gould,", 74, Australian activist and bookseller.",https://en.wikipedia.org/wiki/Bob_Gould_(activist),15,2011,May,None,74.0,None,Australia,None,None,2.772589,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,1,0,0,1,0,0,0,2
74873,20,Mira Zakai,", 76, Israeli opera singer, complications from a stroke.",https://en.wikipedia.org/wiki/Mira_Zakai,7,2019,May,None,76.0,None,Israel,None,None,2.079442,complications from a stroke,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,1,0,0,0,0,0,0,1
66363,2,Mundell Lowe,", 95, American jazz guitarist and composer.",https://en.wikipedia.org/wiki/Mundell_Lowe,6,2017,December,None,95.0,None,United States of America,None,None,1.945910,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,1,0,0,0,0,0,0,1
55769,26,Jerrold Kemp,", 94, American academic.",https://en.wikipedia.org/wiki/Jerrold_Kemp,12,2015,November,None,94.0,None,United States of America,None,None,2.564949,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,1,0,0,0,0,0,0,0,0,1
82010,2,Jacques Noyer,", 93, French Roman Catholic prelate, Bishop of Amiens .",https://en.wikipedia.org/wiki/Jacques_Noyer,5,2020,June,,93.0,None,France,Italy,1987 2003,1.791759,Bishop of Amiens,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,1,0,0,0,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

### Checking Data Types, Duplicates, and Null Values

In [5]:
# Checking data types and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98041 entries, 0 to 98040
Data columns (total 44 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   day                        98041 non-null  object 
 1   name                       98041 non-null  object 
 2   info                       98041 non-null  object 
 3   link                       98041 non-null  object 
 4   num_references             98041 non-null  int64  
 5   year                       98041 non-null  int64  
 6   month                      98041 non-null  object 
 7   info_parenth               36660 non-null  object 
 8   age                        98041 non-null  float64
 9   cause_of_death             17 non-null     object 
 10  place_1                    97888 non-null  object 
 11  place_2                    8115 non-null   object 
 12  info_parenth_copy          36660 non-null  object 
 13  log_num_references         98041 non-null  flo

<IPython.core.display.Javascript object>

#### Observations:
- With our dataset loaded, we can pick up where we left off with extracting known_for values by rebuilding `known_for_dict` and starting the search of `info_3` columns.
- We will need to adjust our approach at this step, in order to also capture `cause_of_death` values.
- Prior to this point we have hard-coded the few `cause_of_death` values encountered, but we expect a much higher proportion of them for the remaining numbered columns.
- We will add a new list `cause_of_death` to collect those values and we will add a separate loop to extract them to the `cause_of_death` column.

### Extracting Remaining `known_for` and `cause_of_death` Values

#### Finding `known_for` Roles and `cause_of_death` in `info_3_0`

In [6]:
# # Obtaining values for column and their counts
# roles_cause_list = df["info_3_0"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [7]:
# # Code to check each value
# roles_cause_list.pop()

<IPython.core.display.Javascript object>

In [8]:
# # Create specific_roles_cause_list for above popped value
# specific_roles_cause_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_3_0"].notna()].index
#             if "shot" in df.loc[index, "info_3_0"]
#         ],
#         "info_3_0",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [9]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_cause_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [10]:
# # Example code to quick-check a specific entry
# df[df["info_3_0"] == "shot Eddie Waitkus"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category and for `cause_of_death`

In [11]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = [
    "campaigner for breast cancer drug Herceptin",
    "MPP of the Legislative Assembly of for Beaches—Woodbine",
    "MP for Secretary General of the Council of and",
    "MPP of the Legislature for Waterloo North",
    "MP of the Karnataka Legislative Assembly",
    "MP of the Odisha Legislative Assembly",
    "MP for Medicine Hat—Cardston—Warner",
    "MP for Newcastle upon Tyne Central",
    "MP for Newcastle upon Tyne North",
    "MP for Perth—Wellington—Waterloo",
    "MP for North West Leicestershire",
    "MP for Northumberland—Miramichi",
    "MP for Wolverhampton South East",
    "MP for Newcastle upon Tyne East",
    "MP for South East St Elizabeth",
    "MP for North Central Clarendon",
    "MP for Manchester Wythenshawe",
    "first honorary MPLA President",
    "MP for Movement of the People",
    "MP for Beauharnois—Salaberry",
    "MP for Birmingham Perry Barr",
    "MP for Birmingham Handsworth",
    "MP for Birmingham Hall Green",
    "MP for Scarborough—Agincourt",
    "MP for Northumberland—Durham",
    "MP for Battleford—Kindersley",
    "MP for Birmingham Northfield",
    "MP for Sheffield Brightside",
    "MP for Leicester North West",
    "MP for City of Buenos Aires",
    "Queensland MP for Mackenzie",
    "MP for Amman fifth district",
    "MP for Stoke on Trent South",
    "MP for Leicester South East",
    "MP for Manchester Blackley",
    "MP for Windsor—Walkerville",
    "MP for Carleton Gloucester",
    "MP for Manchester Openshaw",
    "MP for Coventry North East",
    "MP for Croydon North West",
    "MP for East Aberdeenshire",
    "MP for Vancouver Kingsway",
    "MP for Birmingham Yardley",
    "MP for Nottingham Central",
    "MP for Afram Plains North",
    "MP for Bristol North West",
    "MP for Wandsworth Central",
    "MP for West Aberdeenshire",
    "MP for Manchester Ardwick",
    "MP for Manchester Central",
    "Conservative MP for South",
    "MP for Glasgow Queen Park",
    "MP for Weston super Mare",
    "MP for East Renfrewshire",
    "MP for Ambunti Dreikikir",
    "MP for Barrow in Furness",
    "MP for Mission—Coquitlam",
    "federal MP for St George",
    "MP for Belfast Shankill",
    "MPP for Dufferin Simcoe",
    "MPP for Wentworth North",
    "MP for Sheffield Heeley",
    "MP for Glasgow Cathcart",
    "MP for Clwyd North West",
    "co developer of the IMP",
    "MP for Paddington North",
    "MP for Bouches du Rhône",
    "MP for Grenville—Dundas",
    "MP for Bournemouth East",
    "MP for Stockholm County",
    "MP for Southampton West",
    "MP for Glenrothes since",
    "MP for Mid Bedfordshire",
    "MP for Brome—Missisquoi",
    "former MPP for Kingston",
    "MP for South Manchester",
    "MP for Sheffield Hallam",
    "MP for Naogaon District",
    "MP for Hemel Hempstead",
    "Khyber Pakhtunkhwa MPA",
    "MP for South West Nova",
    "MP for Bury St Edmunds",
    "MP for Bishop Auckland",
    "MP for Nottingham West",
    "MP for Ottawa—Carleton",
    "MP for Montgomeryshire",
    "MP for Coimbatore East",
    "MP for Kingsford Smith",
    "MP for The Battlefords",
    "former Labour Party MP",
    "MP for Blackpool South",
    "MP for Leeds—Grenville",
    "MP for Sault Ste Marie",
    "MP for Central Honiara",
    "NSW MP for Burrinjuck",
    "MP for Wellingborough",
    "MP for City of Durham",
    "MP for North Cornwall",
    "MP for Sarnia—Lambton",
    "MPP for Niagara Falls",
    "MP for Hatay Province",
    "MP for Aberdeen South",
    "MP for Knowsley South",
    "MPP of from Cambridge",
    "MP for Stockton North",
    "MP for Winnipeg South",
    "MP for Dalarna County",
    "MP for Brandon—Souris",
    "MP for Glasgow Pollok",
    "MPP for Ottawa Rideau",
    "speaker of MPR DPR in",
    "MP for Uppsala County",
    "MP for Berettyóújfalu",
    "MP for Glasgow Provan",
    "MP for Ikaroa Rāwhiti",
    "MP for Bradford West",
    "MP for Hamilton West",
    "Labour Party list MP",
    "MP for Otago Central",
    "MP for Ottawa—Vanier",
    "MP for Dunedin North",
    "MP for Pas de Calais",
    "MP for Kajiado North",
    "MP for Lewisham West",
    "Labour MP for Newark",
    "MP for Edmonton East",
    "MP for Middlesbrough",
    "MP for Glasgow Govan",
    "MP for North Malaita",
    "MP for Chennai North",
    "MPP for Durham West",
    "MP for South Antrim",
    "MP for Vaipae Tautu",
    "MP for Swansea West",
    "MP for Newport West",
    "MP for North Sydney",
    "NSW MP for Corrimal",
    "MP for Invercargill",
    "MP for Western Hutt",
    "MP for West Lothian",
    "MP for Cardiff West",
    "MP for North Imenti",
    "MPP for Essex South",
    "MP for Rowley Regis",
    "MP for Gelang Patah",
    "MP for Guruve South",
    "MP for Gainsborough",
    "MP for Basingstoke",
    "MP for Cooch Behar",
    "MP for North Shore",
    "MP for Eden Monaro",
    "MP for Oldham West",
    "MP for West Dorset",
    "MP for Temotu Pele",
    "MP for Maharashtra",
    "head of the UOC MP",
    "MP for Capricornia",
    "MP from Saint John",
    "MP for Londonderry",
    "MP for Regina East",
    "MP for West Tyrone",
    "MP for Mount Royal",
    "MP for Harrow West",
    "MP for Grey—Simcoe",
    "MP for Port Arthur",
    "MP for Bexleyheath",
    "MP for Marijampolė",
    "MP for Fredericton",
    "MP for Hull—Aylmer",
    "MP for Wythenshawe",
    "MP for Clackmannan",
    "President of AMPAS",
    "MP for North Devon",
    "MP for Essex South",
    "MP for Banaskantha",
    "MP for Billericay",
    "MP for Guanajuato",
    "MP for Banffshire",
    "MP for Eastbourne",
    "MP for West Derby",
    "MP for Heretaunga",
    "Northern Cape MPL",
    "MP for Repentigny",
    "MP for Bromsgrove",
    "MP for Accrington",
    "MP for Island Bay",
    "MP for Clydesdale",
    "MP for The Wrekin",
    "MP for Vijayawada",
    "MP for Leominster",
    "MP for Kilmarnock",
    "MP for Hull North",
    "MP for Carshalton",
    "MP for Midlothian",
    "MP for York North",
    "MP for Rushcliffe",
    "MPP for Bellwoods",
    "MP for South Down",
    "MP for Mahasamund",
    "MP for Deggendorf",
    "MP for Buckingham",
    "MP for Coimbatore",
    "MP for Nuevo León",
    "MP for Charlevoix",
    "MP for Srikakulam",
    "MP for Pontefract",
    "MP for Winchester",
    "MPP for York East",
    "MP for Brentford",
    "MP for Wairarapa",
    "MP for Lyttelton",
    "MP for Faversham",
    "MP for Warringah",
    "MP for Cambridge",
    "MP for Orpington",
    "MP for Rochester",
    "MP for Frontenac",
    "MP for Hampstead",
    "MP for Szigetvár",
    "MP for Tongariro",
    "MP for Churchill",
    "MP for Easington",
    "MP for Kaohsiung",
    "MP for Brighouse",
    "MP for Greenwich",
    "MP for Tongatapu",
    "MP for Robertson",
    "MP for Waitakere",
    "MP for Pencarrow",
    "MP for Worcester",
    "MP for Stretford",
    "MP for Smethwick",
    "MP for Tottenham",
    "MP for Nagercoil",
    "MP for Nizamabad",
    "MP for Lancaster",
    "MP for Cuddalore",
    "MP for Ashburton",
    "MP for Wakefield",
    "MP for Penistone",
    "MP for Saarlouis",
    "MP for Mt Albert",
    "MP for Hastings",
    "MP for Richmond",
    "MP for Barnsley",
    "MP for Coahuila",
    "MP for La Trobe",
    "MP for Wide Bay",
    "MP for Falmouth",
    "MP for McMillan",
    "MP for Galloway",
    "MP for Värmland",
    "Balochistan MPA",
    "MP for Waterloo",
    "MP for Solihull",
    "MP for Hereford",
    "MP for Greenock",
    "MP for Hyndburn",
    "MP for El Koura",
    "MP for Palliser",
    "MP for Finchley",
    "MP for Tiverton",
    "MP for Sherwood",
    "MP for Bilaspur",
    "MP for Jelutong",
    "MP for Fallujah",
    "MP for Keighley",
    "MP for Chittoor",
    "MPP for Welland",
    "MP for Heywood",
    "MP for Dalarna",
    "MP for Denison",
    "MP for Badulla",
    "MP for Lasalle",
    "MP for Jalisco",
    "MP for Makueni",
    "MP for Bangaon",
    "MP for Western",
    "MP for Entally",
    "MP for Nyakach",
    "MP for Newbury",
    "MP for Taunton",
    "MP for Dum Dum",
    "MP for Feltham",
    "MP for Mitcham",
    "MP for Wallace",
    "MP of Northern",
    "independent MP",
    "MP for Blaydon",
    "MP for Trinity",
    "MP for Consett",
    "MP for Romford",
    "MP for Spadina",
    "MP for Mercier",
    "MP for Sudbury",
    "MP for Ipswich",
    "MP for Oxford",
    "Queensland MP",
    "MP for Jorhat",
    "MP for Ukonga",
    "MP for Hughes",
    "MP for Dungun",
    "MP for Rompin",
    "MPP for South",
    "MP for Dawson",
    "MP for Argyll",
    "three time MP",
    "MP for Halton",
    "MP of Gujarat",
    "MP for Kabete",
    "MP for Belper",
    "MP for Oaxaca",
    "MP for Melton",
    "MP for Boston",
    "MP for Conway",
    "MP for Mannar",
    "MP for Kigoma",
    "MP for Khulna",
    "MP for Araria",
    "MP for Batley",
    "MP for Leyton",
    "MP for Bowman",
    "MP for Kenema",
    "MP for Jarrow",
    "MP for Ngella",
    "MP for Maldon",
    "MP for Gwydir",
    "MP for Butere",
    "MP for Ndhiwa",
    "MP for Brecon",
    "MP for Fraser",
    "MP for Hunter",
    "MP for Heston",
    "MP for Athens",
    "MP for Viborg",
    "MP for Tumkur",
    "MP for Dudley",
    "MP for Kapiti",
    "MP for Howrah",
    "MP for Napier",
    "MP for Henty",
    "MP for Poole",
    "MP for Geita",
    "New South MP",
    "MP for Brant",
    "MP for Ranau",
    "MP for Waipa",
    "MP for Alwar",
    "MP for Acton",
    "MP for Perth",
    "MP for Truro",
    "MP for Royal",
    "MP for Udupi",
    "MP for Wells",
    "MP for Conwy",
    "MP for Elgin",
    "MP for Nketa",
    "MP for Monor",
    "five time MP",
    "MP for Swan",
    "MP for Swat",
    "MP of Sabah",
    "MP for Sibu",
    "Gauteng MPL",
    "MP for Raub",
    "MP for Hutt",
    "MP for Lowe",
    "Liberal MPP",
    "MP for Aska",
    "MP for York",
    "MP for Buem",
    "Punjab MPA",
    "MP for Bow",
    "MP for Ayr",
    "Sindh MPA",
    "List MP",
    "NSW MP",
    "MPP of",
    "MP for",
    "MPP",
    "MPA",
    "MPL",
    "MP",
]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "shot Andy Warhol Shot Marilyns paintings",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = [
    "and Olympic shot putter",
    "shot putter",
]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = [
    "known as one of the world leading authorities on cancer research",
    "co developer of ultrasound use in cancer detection",
    "treated herself for breast cancer on Antarctica in",
    "co discoverer of drugs that fight cancer",
    "pioneer in breast cancer treatment",
    "expert in breast cancer treatment",
]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = []
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = []
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = []
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = []
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = [
    "pediatric cancer advocate",
]
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = []
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = [
    "shot Eddie Waitkus",
]
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = []
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

cause_of_death = [
    "stroke following decade long battle with breast cancer",
    "died during treatment for testicular cancer in",
    "kidney failure associated with colon cancer",
    "complications of treatment for lung cancer",
    "pneumonia as a complication of lung cancer",
    "meningitis complicated from breast cancer",
    "complications from kidney cancer surgery",
    "complications of breast cancer treatment",
    "complications of prostate cancer surgery",
    "complications from bowel cancer surgery",
    "complications from gall bladder cancer",
    "pancreatic cancer",
    "heart attack due to pancreatic cancer",
    "complications from bone marrow cancer",
    "pneumonia as a complication of cancer",
    "complication following cancer surgery",
    "complications from esophageal cancer",
    "complications from pancreatic cancer",
    "complications from colorectal cancer",
    "heart attack during cancer treatment",
    "complications from bile duct cancer",
    "complications from prostate cancer",
    "cardiac arrest due to colon cancer",
    "as a complication of breast cancer",
    "complications of colorectal cancer",
    "complications of pancreatic cancer",
    "complications from bladder cancer",
    "complications from stomach cancer",
    "complications from cancer surgery",
    "liver cancer complicated by COVID",
    "complications from throat cancer",
    "complications from breast cancer",
    "complications of prostate cancer",
    "kidney complications from cancer",
    "complications from liver cancer",
    "complications from colon cancer",
    "complications from brain cancer",
    "complications of ovarian cancer",
    "euthanised for abdominal cancer",
    "complications from lung cancer",
    "complications of breast cancer",
    "complications from skin cancer",
    "complications from oral cancer",
    "complications of brain cancer",
    "complications of liver cancer",
    "complications of colon cancer",
    "complications of lung cancer",
    "metastatic pancreatic cancer",
    "metastatic esophageal cancer",
    "throat cancer",
    "complications from cancer",
    "anaplastic thyroid cancer",
    "urothelial bladder cancer",
    "metastasized liver cancer",
    "metastatic breast cancer",
    "medullary thyroid cancer",
    "euthanized due to cancer",
    "cancer related pneumonia",
    "complications of cancer",
    "metastatic colon cancer",
    "gastrointestinal cancer",
    "small cell lung cancer",
    "small intestine cancer",
    "neuroendrocrine cancer",
    "cancer related illness",
    "salivary gland cancer",
    "neuroendocrine cancer",
    "hepatocellular cancer",
    "metastatic eye cancer",
    "cancerous peritonitis",
    "periampullary cancer",
    "spinal fluid cancer",
    "cancer of the spine",
    "renal pelvic cancer",
    "bone marrow cancer",
    "gallbladder cancer",
    "oesophageal cancer",
    "endometrial cancer",
    "male breast cancer",
    "nasopharynx cancer",
    "pancreatic cancer",
    "esophageal cancer",
    "colorectal cancer",
    "intestinal cancer",
    "testicular cancer",
    "peritoneal cancer",
    "pharyngeal cancer",
    "metastatic cancer",
    "mandibular cancer",
    "Leka Zogu; cancer",
    "bile duct cancer",
    "laryngeal cancer",
    "abdominal cancer",
    "lymphatic cancer",
    "esophagus cancer",
    "carcinoid cancer",
    "ampullary cancer",
    "prostate cancer",
    "cervical cancer",
    "appendix cancer",
    "pancreas cancer",
    "lymphoma cancer",
    "thoracic cancer",
    "urethral cancer",
    "stomach cancer",
    "ovarian cancer",
    "bladder cancer",
    "uterine cancer",
    "thyroid cancer",
    "adrenal cancer",
    "gastric cancer",
    "myeloma cancer",
    "vaginal cancer",
    "of lung cancer",
    "throat cancer",
    "kidney cancer",
    "spinal cancer",
    "tongue cancer",
    "tonsil cancer",
    "thymic cancer",
    "rectal cancer",
    "vulvar cancer",
    "thymus cancer",
    "pelvic cancer",
    "liver cancer",
    "brain cancer",
    "colon cancer",
    "bowel cancer",
    "blood cancer",
    "renal cancer",
    "heart cancer",
    "sinus cancer",
    "mouth cancer",
    "spine cancer",
    "Liver cancer",
    "lung cancer",
    "bone cancer",
    "skin cancer",
    "oral cancer",
    "anal cancer",
    "nose cancer",
    "heart attack following a cerebral haemorrhage",
    "heart attack during Olympic marathon trials",
    "suspected heart attack while hillwalking",
    "heart attack caused by anorexia nervosa",
    "cardiac arrest following a heart attack",
    "complications following a heart attack",
    "heart attack following a hunger strike",
    "heart attack brought about by diabetes",
    "heart attack caused by a drug overdose",
    "suffered a heart attack while swimming",
    "complications following a\xa0heart attack",
    "heart attack following spinal surgery",
    "heart attack due to pancreatic cancer",
    "apparent heart attack while teaching",
    "heart failure following heart attack",
    "brain haemorrhage after heart attack",
    "heart attack during cancer treatment",
    "complications from a heart attack",
    "heart attack triggered by asthma",
    "complications from heart attack",
    "complications of a heart attack",
    "heart attack following beating",
    "complications of heart attack",
    "series of small heart attacks",
    "heart attack aboard aircraft",
    "heart attack as a result of",
    "heart attack while jogging",
    "post surgery heart attack",
    "heart attack after race",
    "suspected heart attack",
    "apparent heart attack",
    "probable heart attack",
    "possible heart attack",
    "heart attack",
    "heart failure resulting from hypertrophic cardiomyopathy",
    "heart failure as a complication from cardiac surgery",
    "heart failure after surgery following a knockout",
    "heart failure after a botched suicide attempt",
    'known as "Crazy" Luke Graham; heart failure',
    "complications from congestive heart failure",
    "heart failure following accidental overdose",
    "heart failure caused by anorexia nervosa",
    "heart failure due to pulmonary embolism",
    "heart failure due to cardiac arrhythmia",
    "complications related to heart failure",
    "heart failure following heart attack",
    "pneumonia congestive heart failure",
    "heart failure related to pneumonia",
    "heart failure due to polymyositis",
    "complications from heart failure",
    "complications of heart failure",
    "progeria related heart failure",
    "hypertensive heart failure",
    "heart failure from sepsis",
    "congestive heart failure",
    "suspected heart failure",
    "apparent heart failure",
    "acute heart failure",
    "dheart failure",
    "heart failure",
    "liver failure reportedly complicated by COVID",
    "pulmonary fibrosis complicated by COVID",
    "cardiovascular illness related to COVID",
    "Parkinson disease complicated by COVID",
    "multiple organ failure caused by COVID",
    "kidney problems aggravated by COVID",
    "renal failure complicated by COVID",
    "kidney failure brought on by COVID",
    "sepsis as a complication of COVID",
    "long illness complicated by COVID",
    "liver cancer complicated by COVID",
    "pneumonia complicated by COVID",
    "heart complications from COVID",
    "viral pneumonia from COVID",
    "cardiac arrest from COVID",
    "complications from COVID",
    "COVID related pneumonia",
    "complications of COVID",
    "post COVID pneumonia",
    "pneumonia from COVID",
    "COVID",
    "hospitalized with pneumonia since his evacuation several days after",
    "pneumonia as a complication from a kidney infection",
    "pneumonia with complications from Alzheimer disease",
    "pneumonia as a complication of multiple myeloma",
    "pneumonia induced corticobasal degeneration",
    "pneumonia as a complication of lung cancer",
    "pneumonia as a complication from a stroke",
    "complications of pneumonia from surgery",
    "pneumonia as a complication of a stroke",
    "pneumonia as a complication of cancer",
    "bronchopneumonia following a stroke",
    "heart failure related to pneumonia",
    "pneumonia congestive heart failure",
    "respiratory failure from pneumonia",
    "complications following pneumonia",
    "complications of viral pneumonia",
    "pneumonia complicated by COVID",
    "complications from pneumonia",
    "pneumonia following a stroke",
    "complication from pneumonia",
    "complications of pneumonia",
    "viral pneumonia from COVID",
    "pneumonia related illness",
    "cancer related pneumonia",
    "COVID related pneumonia",
    "AIDS related pneumonia",
    "interstitial pneumonia",
    "aspiration pneumonia",
    "pneumonia from COVID",
    "hypostatic pneumonia",
    "post COVID pneumonia",
    "bronchial pneumonia",
    "bilateral pneumonia",
    "pleural pneumonia",
    "bronchopneumonia",
    "double pneumonia",
    "pneumonia during",
    "viral pneumonia",
    "acute pneumonia",
    "pneumonia",
    "stroke following decade long battle with breast cancer",
    "stroke as a complication of an aortic aneurysm",
    "cardiac arrest as a complication from a stroke",
    "stroke during treatment of pulmonary embolism",
    "stroke related to acute myelogenous leukemia",
    "stroke as a complication from heart surgery",
    "pneumonia as a complication from a stroke",
    "stroke from vertebral artery dissection",
    "pneumonia as a complication of a stroke",
    "complications from a series of strokes",
    "complications following a heat stroke",
    "complications of a stroke suffered in",
    "complications from multiple strokes",
    "bronchopneumonia following a stroke",
    "declining health following stroke",
    "complications following a stroke",
    "complications of massive stroke",
    "complications following strokes",
    "complications from heat stroke",
    "complications from heatstroke",
    "following a series of strokes",
    "cerebral atrophy from stroke",
    "pneumonia following a stroke",
    "complications after a stroke",
    "complications from a stroke",
    "aneurysm following a stroke",
    "complications from strokes",
    "complications of a stroke",
    "complications from stroke",
    "complications of strokes",
    "hemorrhagic brain stroke",
    "complications of stroke",
    "stroke following a fall",
    "consequences of stroke",
    "stroke complications",
    "died of a stroke in",
    "hemorrhagic stroke",
    "series of strokes",
    "multiple strokes",
    "watershed stroke",
    "cerebral stroke",
    "heat stroke",
    "heatstroke",
    "strokes",
    "stroke",
]
cause_of_death = sorted(list(set(cause_of_death)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [12]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting `known_for` Categories and `cause_of_death` Values from `info_3_0`

In [13]:
%%time

# Column to check
column = 'info_3_0'

# Start dataframe
dataframe = df[df[column].notna()]

# For loop to find cause in column and extract it to cause_of_death
for cause in cause_of_death:
    for index in dataframe.index:
        item = df.loc[index, column]
        if item:
            if cause in item:
                df.loc[index, 'cause_of_death'] = cause
                df.loc[index, column] = item.replace(cause, '').strip()
                
# For loop to find role in column and extract it as category
for category, category_lst in known_for_dict.items():
    for role in category_lst:
        for index in dataframe.index:
                item = df.loc[index, column]
                if item:
                    if role in item:
                        df.loc[index, category] = 1
                        df.loc[index, column] = item.replace(role, '').strip()

# Calculating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking remaining number of missing cause_of_death values
print(f'There are {df["cause_of_death"].notna().sum()} values in cause_of_death column.\n')

There are 10084 values in cause_of_death column.

CPU times: total: 3min 31s
Wall time: 3min 31s


<IPython.core.display.Javascript object>

#### Checking Updated `num_categories` Value Counts

In [14]:
# Checking Updated num_categories Value Counts
df["num_categories"].value_counts()

1    88120
2     9682
3      194
0       45
Name: num_categories, dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` and `cause_of_death` for the next iteration.

#### Finding `known_for` Roles and `cause_of_death` in `info_3_0`

In [15]:
# # Obtaining values for column and their counts
# roles_cause_list = df["info_3_0"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [16]:
# # Code to check each value
# roles_cause_list.pop()

<IPython.core.display.Javascript object>

In [17]:
# # Create specific_roles_cause_list for above popped value
# specific_roles_cause_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_3_0"].notna()].index
#             if "Olymp" in df.loc[index, "info_3_0"]
#         ],
#         "info_3_0",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [18]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_cause_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [19]:
# # Example code to quick-check a specific entry
# df[df["info_3_0"] == "Mr Olympia"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category and for `cause_of_death`

In [20]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = [
    "member of the House of Representatives from Massachusetts th congressional district",
    "member of the House of Representatives from South Dakota nd congressional district",
    "member of the House of Representatives from Wisconsin rd congressional district",
    "member of the House of Representatives from Illinois rd congressional district",
    "member of the House of Representatives from Arkansas th congressional district",
    "member of the House of Representatives from Michigan rd congressional district",
    "member of the House of Representatives from Illinois th congressional district",
    "member of the House of Representatives for Wisconsin th congressional district",
    "member of the House of Representatives from Maryland th congressional district",
    "member of the House of Representatives from Indiana th congressional district",
    "Member of the House of Representatives from Indiana th congressional district",
    "member of the House of Representatives for Alabama th congressional district",
    "member of the House of Representatives from Ohio th congressional district",
    "member of the House of Representatives for Idaho nd congressional district",
    "member of the House of Representatives from Utah st congressional district",
    "having served as a Labour House of Representatives of Member of Parliament",
    "member of the House of Representatives from th congressional district",
    "member of the House of Representatives from st congressional district",
    "member of the House of Representatives for North Carolina th district",
    "Member of the House of Representatives from th congressional district",
    "House of Representatives from Wyoming at large congressional district",
    "member of the House of Representatives for th congressional district",
    "member of the House of Representatives for West Virginia rd district",
    "member of the House of Representatives for Vermont at large district",
    "member of the House of Representatives from Tennessee th district",
    "member of the House of Representatives from Virginia th district",
    "member of the House of Representatives from Arkansas rd district",
    "member of the House of Representatives for Minnesota th district",
    "member of the House of Representatives for Minnesota th District",
    "member of the House of Representatives for Maryland nd district",
    "member of the House of Representatives for Missouri th district",
    "Permanent Representative to the United Nations Office at Geneva",
    "member of the House of Representatives for Illinois nd district",
    "member of the House of Representatives from Montana nd district",
    "member of the House of Representatives for Maryland th District",
    "member of the House of Representatives from Hawaii nd district",
    "member of the House of Representatives for Alabama th district",
    "member of the House of Representatives from Nevada nd district",
    "four time member of the New Hampshire House of Representatives",
    "member of the House of Representatives for Hawaii st district",
    "longest serving female member of the House of Representatives",
    "member of the Jakarta Regional People Representative Council",
    "member of the House of Representatives from Iowa rd district",
    "member of the House of Representatives for Ohio th district",
    "Representative from West Virginia th congressional district",
    "member of the House of Representatives from New th district",
    "member of the House of Representatives for the st District",
    "member of the House of Representatives for New th district",
    "member of the House of Representatives from th districts",
    "Representative from the th District of Negros Occidental",
    "member of the House of Representatives from th district",
    "member of the House of Representatives from Michigan th",
    "member of the House of Representatives from Arkansas th",
    "member of the House of Representatives for Wisconsin th",
    "member of the House of Representatives from Illinois nd",
    "Representative from Tennessee rd congressional district",
    "member of the House of Representatives from Connecticut",
    "member of the House of Representatives for th district",
    "member of House of Representatives for the st district",
    "member of the House of Representatives from Arizona nd",
    "former Republican Representative from Illinois from to",
    "former Republican Representative from Michigan from to",
    "member of the House of Representatives for st district",
    "member of the North Carolina House of Representatives",
    "member of the House of Representatives from Minnesota",
    "Member of the North Carolina House of Representatives",
    "former Democratic Representative from Washington from",
    "former Republican Representative from Colorado served",
    "Speaker of the Massachusetts House of Representatives",
    "member of the Massachusetts House of Representatives",
    "member of the New Hampshire House of Representatives",
    "member of the House of Representatives from Colorado",
    "member of the House of Representatives of the Senate",
    "member of the House of Representatives for St George",
    "member of the House of Representatives from Michigan",
    "member of the House of Representatives from Missouri",
    "member of the House of Representatives from Illinois",
    "member of the South Dakota House of Representatives",
    "member of the North Dakota House of Representatives",
    "member of the House of Representatives from Indiana",
    "Member of the South Dakota House of Representatives",
    "former Republican Representative from Maine from to",
    "member of the House of Representatives for Illinois",
    "member of the Connecticut House of Representatives",
    "member of the House of Representatives for Gilmore",
    "member of the House of Representatives from Oregon",
    "member of the House of Representatives for Batanes",
    "member of the House of Representatives for Ohio st",
    "delegate to the House of Representatives from Guam",
    "member of the House of Representatives for Forrest",
    "Permanent Representative to the Economic Community",
    "member of the House of Representatives for Berowra",
    "member of the House of Representatives for Phillip",
    "member of the Washington House of Representatives",
    "member of House of Representatives from Wisconsin",
    "member of the House of Representatives for Cowper",
    "member of the House of Representatives for Sydney",
    "member of the House of Representatives for Oregon",
    "speaker of the Tennessee House of Representatives",
    "Speaker of the Tennessee House of Representatives",
    "member of North Carolina House of Representatives",
    "member of the Minnesota House of Representatives",
    "member of the Tennessee House of Representatives",
    "member of the Missouri Houses of Representatives",
    "member of the House of Representatives from Ohio",
    "member of the House of Representatives from Utah",
    "Speaker of the Missouri House of Representatives",
    "member of the House of Representatives for Bohol",
    "member of the Illinois House of Representatives",
    "member of the Missouri House of Representatives",
    "member of the Michigan House of Representatives",
    "member of the Arkansas House of Representatives",
    "member of the Oklahoma House of Representatives",
    "member of the Colorado House of Representatives",
    "member of the Delaware House of Representatives",
    "Senator of the Indiana House of Representatives",
    "Permanent Representative of Polisario to the UN",
    "member of the House of Representatives for Reid",
    "member of the House of Representatives from New",
    "former liberal Democrat Representative from and",
    "former Republican Representative from Iowa from",
    "member of the House of Representatives for Ohio",
    "last Doorkeeper of the House of Representatives",
    "member of South Dakota House of Representatives",
    "member of the Alabama House of Representatives",
    "member of the Wyoming House of Representatives",
    "member of the Arizona House of Representatives",
    "member of the Indiana House of Representatives",
    "member of the Vermont House of Representatives",
    "member of the Montana House of Representatives",
    "member of the House of Representatives from th",
    "Permanent Representative to the United Nations",
    "Speaker of the Hawaii House of Representatives",
    "senator of the Alaska House of Representatives",
    "Member of the Vermont House of Representatives",
    "member of the House of Representatives from rd",
    "Member of the House of Representatives from th",
    "first Delegate to the House of Representatives",
    "former Democratic Representative from Michigan",
    "Speaker of the Alaska House of Representatives",
    "member of the Alaska House of Representatives",
    "member of the Kansas House of Representatives",
    "member of the Oregon House of Representatives",
    "member of the Hawaii House of Representatives",
    "member of the House of Representatives for th",
    "former member of the House of Representatives",
    "member of the Nevada House of Representatives",
    "Member of the Hawaii House of Representatives",
    "member of the Idaho House of Representatives",
    "member of the Maine House of Representatives",
    "Member of Minnesota House of Representatives",
    "member of the House of Representatives since",
    "Democratic Congressional Representative from",
    "Speaker of the Utah House of Representatives",
    "member of the Ohio House of Representatives",
    "member of the Iowa House of Representatives",
    "member of the Utah House of Representatives",
    "member of the House of Representatives from",
    "Member of the Ohio House of Representatives",
    "member of the People Representative Council",
    "member of Oklahoma House of Representatives",
    "husband of Representative Carolyn B Maloney",
    "member of the New House of Representatives",
    "member of the P R House of Representatives",
    "former Republican Representative from from",
    "member of the House of Representatives for",
    "member of Wyoming House of Representatives",
    "president of the House of Representatives",
    "member of Hawaii House of Representatives",
    "Representative from Minnesota th district",
    "member of the House of Representatives of",
    "Historian of the House of Representatives",
    "member of the Chamber of Representatives",
    "member of the Council of Representatives",
    "member of Maine House of Representatives",
    "Representative from Illinois th district",
    "Delegate to the House of Representatives",
    "first Permanent Representative to the UN",
    "Speaker of the House of Representatives",
    "speaker of the House of Representatives",
    "member of the Houses of Representatives",
    "member of House of Representatives from",
    "member of the House of Representatives",
    "Member of the House of Representatives",
    "Massachusetts House of Representatives",
    "former Democratic Representative from",
    "former Republican Representative from",
    "Speaker of House of Representatives",
    "member of House of Representatives",
    "Representative from North Carolina",
    "Tennessee House of Representatives",
    "Representative from New Hampshire",
    "Permanent Representative to NATO",
    "Representative from North Dakota",
    "Representative from South Dakota",
    "Representative from Connecticut",
    "Representative for Connecticut",
    "Republican Representative from",
    "Representative from Wisconsin",
    "Utah House of Representatives",
    "Representative from Tennessee",
    "Arkansas State Representative",
    "Colorado State Representative",
    "Representative from Illinois",
    "Representative from Virginia",
    "Representative from Michigan",
    "Representative from Missouri",
    "Representative from Maryland",
    "Representative from Indiana",
    "Representative Co Prince of",
    "Representative for Michigan",
    "Representative from Arizona",
    "Representative from Alabama",
    "Representative from Vermont",
    "Kansas State Representative",
    "Representative for Virginia",
    "Representative from Hawaii",
    "Representative from Alaska",
    "Representative from Oregon",
    "Representative from Kansas",
    "Representative from Maine",
    "Representative from Ohio",
    "Representative from Iowa",
    "Representative from New",
    "Representative for Ohio",
    "Representative for Utah",
    "Representative for Iowa",
    "Arkansas Representative",
    "State Representative",
    "Representative elect",
    "Trade Representative",
    "Queen Representative",
    "Representative from",
    "Representative for",
    "Representative",
    "co writer of the Endangered Species Act of",
]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "first female writer awarded full press credentials at",
    "member of the Nashville Songwriters Hall of Fame",
    "a prolific writer of language text books",
    "widow of science fiction writer",
    "award winning Ontarian writer",
    "and television screenwriter",
    "presidential speechwriter",
    "speech writer",
    "detective story writer",
    "songwriter arranger",
    "nationalized writer",
    "short story writer",
    "esotericism writer",
    "non‑fiction writer",
    "singer songwriter",
    "television writer",
    "songwriter winner",
    "technical writer",
    "cookbook writer",
    "children writer",
    "dialogue writer",
    "fiction writer",
    "and songwriter",
    "travel writer",
    "script writer",
    "comics writer",
    "comedy writer",
    "screenwriter",
    "scriptwriter",
    "music writer",
    "story writer",
    "food writer",
    "songwriter",
    "and writer",
    "art writer",
    "Director General of the Olympia",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = [
    "only Olympian handball player to represent three countries",
    "won the gold medal in hurdles at the Summer Olympics",
    "winner of seven Olympic gold medals for the Union",
    "president of the International Olympic Committee",
    "bronze medalist in the Summer Olympics marathon",
    "International Olympic Committee whistleblower",
    "first black woman to become Olympic champion",
    "national team member for the Winter Olympics",
    "President of the National Olympic Committee",
    "most medaled athlete at the Summer Olympics",
    "gold medallist in the Winter Olympic Games",
    "winner of the Olympic demonstration event",
    "led push to return tennis to Olympics in",
    "represented in soccer at Summer Olympics",
    "brother of Olympic Judo star Neil Adams",
    "silver medallist at the Summer Olympics",
    "silver medalist at the Summer Olympics",
    "gold medallist at the Summer Olympics",
    "Olympic gold medallist for the Union",
    "lit cauldron at the Summer Olympics",
    "president of Olympique de Marseille",
    "president of the Olympic Committee",
    "represented in six Chess Olympiads",
    "longest living Olympic competitor",
    "Chairman of the Olympic Committee",
    "gold medallist at Summer Olympics",
    "oldest known Olympic medal winner",
    "two time Olympic silver medalist",
    "Olympic gold medal winning boxer",
    "double medallist at the Olympics",
    "Olympics high jump gold medalist",
    "Winter Olympics silver medalist",
    "three time Olympic medal winner",
    "founder of the Special Olympics",
    "four time Olympic gold medalist",
    "Summer Olympics silver medalist",
    "president of Olympic Committee",
    "Olympic double silver medalist",
    "oldest living Olympic champion",
    "first female Olympic champion",
    "seven time Olympic competitor",
    "Youth Olympic silver medalist",
    "Olympic middleweight champion",
    "Olympic bronze medal winner",
    "three time Olympic champion",
    "Olympic silver medal winner",
    "twice Olympic gold medalist",
    "Olympic champion in javelin",
    "Olympic champion under \xa0kg",
    "first Olympic medal winner",
    "multiple Olympic medallist",
    "four time Olympic champion",
    "two time Olympic champion",
    "fourfold Olympic champion",
    "Olympic bronze medallist",
    "Olympic silver medallist",
    "Olympic silver medalist",
    "Olympic bronze medalist",
    "double Olympic champion",
    "triple Olympic champion",
    "Olympic gold medallist",
    "member of Olympic team",
    "Olympic sports shooter",
    "Youth Olympic champion",
    "Olympics gold medalist",
    "Olympic relay champion",
    "Olympic gold medalist",
    "Olympic wrestler for",
    "Olympic silver medal",
    "five time Olympian",
    "Olympic medallist",
    "Olympic champion",
    "Olympic medalist",
    "Olympic Champion",
    "Olympic silver",
    "triple Olympic",
    "Olympic gold",
    "Mr Olympia",
    "Olympian",
]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = [
    "inventor of the implantable cardiac pacemaker",
    "developed the Rho immune globulin vaccine for Rh disease",
    "discoverer of Kawasaki disease",
]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = []
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = []
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = [
    "parachutist at Summer Olympics opening ceremony",
]
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = []
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = [
    "rescued people from suicide",
]
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = [
    "planned Summer Olympics Munich massacre",
]
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = [
    "disguised as female to compete for y at Summer Olympics",
]
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = []
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

cause_of_death = [
    "breast cancer",
    "of cancer",
    "cancer",
    "shot with pepper spray projectile by Boston Police",
    "killed by a head shot together with friend",
    "shot whilst covering the Oaxaca protests",
    "shot during attack on José Ramos Horta",
    "shot down during the Battle of Kyiv",
    "self inflicted gunshot to the head",
    "complications from gunshot wounds",
    "shot in the Rajnandgaon ambush",
    "shot by his Chief of Security",
    "suspected suicide by gunshot",
    "gunshot by Brandon McInerney",
    "shot during domestic dispute",
    "apparent suicide by gunshot",
    "possible suicide by gunshot",
    "shot by the Defense Forces",
    "shot by Special Task Force",
    "shot during prison escape",
    "complications of gunshot",
    "shot by record producer",
    "shot during carjacking",
    "shot by police officer",
    "injuries from gunshot",
    "shot in an army raid",
    "homicide by gunshot",
    "shot during robbery",
    "gunshot to the head",
    "shot by the Army in",
    "suicide by gunshot",
    "accidental gunshot",
    "shot while hunting",
    "shot dead in Basra",
    "shot in East Timor",
    "murder by gunshot",
    "shot in Mogadishu",
    "shot by soldiers",
    "shot by the IRA",
    "shot by robbers",
    "shot by police",
    "gunshot wounds",
    "shot to death",
    "gunshot wound",
    "shot times",
    "shot dead",
    "shot down",
    "gunshot",
    "shot",
    "complications of cerebral hemorrhage from traffic collision",
    "complications from a race collision during Hours of Le Mans",
    "complications from injuries sustained in traffic collision",
    "multiple organ failure following traffic collision",
    "complications from a collision during competition",
    "head injuries sustained in a traffic collision",
    "cardiac arrest due to collision with teammate",
    "head injuries sustained in a race collision",
    "blunt force neck injury from race collision",
    "head injury sustained in traffic collision",
    "injuries sustained in a traffic collision",
    "injuries received in a traffic collision",
    "injuries sustained in traffic collision",
    "cardiac arrest after on field collision",
    "complications from a traffic collision",
    "spinal injuries from traffic collision",
    "injuries sustained in a race collision",
    "complications after traffic collision",
    "complications from traffic collision",
    "motor vehicle collision with a moose",
    "head injuries from traffic collision",
    "injuries from a traffic collision",
    "head injuries from race collision",
    "injuries from traffic collision",
    "injuries sustained in collision",
    "vehicle pedestrian collision",
    "suicide by traffic collision",
    "collision during practice",
    "base jumping collision",
    "motorcycle collision",
    "dirt bike collision",
    "racetrack collision",
    "bobsleigh collision",
    "boat race collision",
    "traffic collision ·",
    "training collision",
    "wingsuit collision",
    "traffic collision",
    "tractor collision",
    "cycling collision",
    "vehicle collision",
    "bicycle collision",
    "balloon collision",
    "plane collision",
    "train collision",
    "stunt collision",
    "race collision",
    "boat collision",
    "ATV collision",
    "car collision",
    "cardiac arrest stemming from decompression sickness",
    "cardiac arrhythmia stemming from atherosclerosis",
    "cardiac arrest due to abdominal aortic aneurysm",
    "cardiac arrest due to collision with teammate",
    "possible cardiac arrest during Dakar Rally",
    "brain hemorrhage following cardiac arrest",
    "cardiac arrest after a spinal cord injury",
    "complications following cardiac surgery",
    "cardiac arrest from respiratory failure",
    "cardiac arrest after on field collision",
    "complications from a cardiac condition",
    "complications following cardiac arrest",
    "cardiac arrest following car accident",
    "complications of a cardiac condition",
    "complications from cardiac surgery",
    "cardiac arrest as a result of COPD",
    "complications from cardiac arrest",
    "complications of cardiac surgery",
    "complications of cardiac arrest",
    "cardiac arrest during surgery",
    "cardiac arrest due to sepsis",
    "cardiac arrest from shooting",
    "pulmonary cardiac arrest",
    "cardiac complications",
    "cardiac arrest due to",
    "cardiac amyloidosis",
    "cardiac dysfunction",
    "cardiac dysrhythmia",
    "cardiac arrhythmia",
    "cardiac arrest and",
    "cardiac arrythmia",
    "cardiac aneurysm",
    "cardiac problems",
    "cardiac failure",
    "cardiac ailment",
    "cardiac illness",
    "cardiac arrest",
    "chronic kidney disease caused by type diabetes",
    "of natural causes after a lengthy illness",
    "respiratory failure caused by brain tumor",
    "discovered cause of sickle cell anemia",
    "liver failure caused by Hepatitis C",
    "brain death caused by knockout",
    "complications caused by AIDS",
    "apparent natural causes",
    "AIDS related causes",
    "natural causes",
    "unknown causes",
    "body found on this date after suicide by carbon monoxide poisoning",
    "possible suicide by overdose of prescription painkillers",
    "and became the West first woman suicide bomber",
    "fall from height ruled a suicide by the police",
    "suspected suicide by carbon monoxide poisoning",
    "suicide by overdose of prescription medication",
    "apparent suicide by carbon monoxide poisoning",
    "suicide by self inflicted blunt force trauma",
    "suspected suicide by fall from a building",
    "suicide by carbon monoxide inhalation",
    "suicide by carbon monoxide poisoning",
    "intercepted suicide bomber at school",
    "assisted suicide by lethal injection",
    "apparent suicide in front of a train",
    "suicide by blunt force head injury",
    "killed by suicide bomb in Northern",
    "suicide by inert gas asphyxiation",
    "suicide by jumping from building",
    "apparent suicide by dehydration",
    "injuries from a suicide attempt",
    "suicide by barbiturate overdose",
    "suicide by self defenestration",
    "suicide by jumping from bridge",
    "suicide by jumping from cliff",
    "suspected suicide by hanging",
    "suspected suicide by gunshot",
    "suicide by alcohol poisoning",
    "suicide by helium inhalation",
    "suicide by traffic collision",
    "suicide by cyanide poisoning",
    "suicide by grenade explosion",
    "apparent suicide by stabbing",
    "apparent suicide by hanging",
    "apparent suicide by gunshot",
    "apparent suicide by jumping",
    "suspected suicide by poison",
    "possible suicide by gunshot",
    "physician assisted suicide",
    "suicide by self immolation",
    "suicide prior to execution",
    "suicide by throat cutting",
    "victim of suicide bombing",
    "suicide by defenestration",
    "suicide by drug overdose",
    "charcoal burning suicide",
    "suicide by strangulation",
    "suicide by wrist cutting",
    "suicide by asphyxiation",
    "suicide by train impact",
    "suicide by hand grenade",
    "suicide by poisoning",
    "suicide by car crash",
    "suicide by drowning",
    "suicide by overdose",
    "suicide bomb attack",
    "suicide by stabbing",
    "suicide bomb victim",
    "suicide by hanging",
    "suicide by gunshot",
    "suicide by jumping",
    "suicide by fasting",
    "officially suicide",
    "suicide by alcohol",
    "suspected suicide",
    "assisted suicide",
    "apparent suicide",
    "suicide by train",
    "suicide by pilot",
    "possible suicide",
    "suicide bombing",
    "suicide by fire",
    "suicide by drug",
    "suicide attack",
    "murder suicide",
    "suicide by",
    "Complications of liver disease",
    "hypertensive atherosclerotic cardiovascular disease",
    "complications of a chronic neurological disease",
    "complications from Charcot Marie Tooth disease",
    "complications from peripheral vascular disease",
    "chronic kidney disease caused by type diabetes",
    "complications of Charcot Marie Tooth disease",
    "complications from polycystic kidney disease",
    "complications related to Alzheimer disease",
    "complications from a neuromuscular disease",
    "complications from cardiovascular disease",
    "lung complications from Alzheimer disease",
    "complications from motor neurone disease",
    "active euthanasia for pulmonary disease",
    "complications related to kidney disease",
    "from complications of Parkinson disease",
    "atherosclerotic cardiovascular disease",
    "complications from Parkinson’s disease",
    "complications from respiratory disease",
    "euthanization following a lung disease",
    "chronic obstructive pulmonary disease",
    "liver disease complicated by diabetes",
    "complications from Parkinsons disease",
    "complications from Alzheimer disease",
    "complications from Parkinson disease",
    "euthanized following adrenal disease",
    "complications from Lewy body disease",
    "f complications of Alzheimer disease",
    "complications from pulmonary disease",
    "complications of respiratory disease",
    "complications of a vascular disease",
    "complications of Parkinson disease",
    "complications of Alzheimer disease",
    "complications due to heart disease",
    "complications of Lewy body disease",
    "complications from kidney disease",
    "complication of Parkinson disease",
    "complications from heart disease",
    "complications from liver disease",
    "pulmonary veno occlusive disease",
    "combination of Parkinson disease",
    "complications from lung disease",
    "complications of kidney disease",
    "gastroesophageal reflux disease",
    "complications of heart disease",
    "arteriosclerotic heart disease",
    "complications of Crohn disease",
    "complications of liver disease",
    "cardiovascular renal disease",
    "hypertensive heart disease",
    "hypertrophic heart disease",
    "degenerative brain disease",
    "Creutzfeldt Jakob disease",
    "interstitial lung disease",
    "neurodegenerative disease",
    "cerebral vascular disease",
    "inflammatory lung disease",
    "acute respiratory disease",
    "chronic pulmonary disease",
    "degenerative lung disease",
    "congenital heart disease",
    "gastrointestinal disease",
    "cerebrovascular disease",
    "coronary artery disease",
    "cardiopulmonary disease",
    "ischaemic heart disease",
    "Erdheim Chester disease",
    "discovered Lyme disease",
    "suspected heart disease",
    "cardiovascular disease",
    "chronic kidney disease",
    "coronary heart disease",
    "ischemic heart disease",
    "motor neurone disease",
    "legionnaires' disease",
    "chronic liver disease",
    "meningococcal disease",
    "motor neuron disease",
    "neurological disease",
    "degenerative disease",
    "Lou Gehrig’s disease",
    "respiratory disease",
    "ebola virus disease",
    "Parkinson’s disease",
    "Ebola virus disease",
    "undisclosed disease",
    "Legionnaire disease",
    "sickle cell disease",
    "Lou Gehrig disease",
    "Huntington disease",
    "infectious disease",
    "autoimmune disease",
    "intestinal disease",
    "Alzheimer disease",
    "Parkinson disease",
    "pulmonary disease",
    "alzheimer disease",
    "vascular disease",
    "coronary disease",
    "Addison disease",
    "kidney disease",
    "muscle disease",
    "heart disease",
    "liver disease",
    "blood disease",
    "renal disease",
    "Crohn disease",
    "nerve disease",
    "brain disease",
    "lung disease",
    "Pick disease",
]
cause_of_death = sorted(list(set(cause_of_death)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [21]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting `known_for` Categories and `cause_of_death` Values from `info_3_0`

In [22]:
%%time

# Column to check
column = 'info_3_0'

# Start dataframe
dataframe = df[df[column].notna()]

# For loop to find cause in column and extract it to cause_of_death
for cause in cause_of_death:
    for index in dataframe.index:
        item = df.loc[index, column]
        if item:
            if cause in item:
                df.loc[index, 'cause_of_death'] = cause
                df.loc[index, column] = item.replace(cause, '').strip()
                
# For loop to find role in column and extract it as category
for category, category_lst in known_for_dict.items():
    for role in category_lst:
        for index in dataframe.index:
                item = df.loc[index, column]
                if item:
                    if role in item:
                        df.loc[index, category] = 1
                        df.loc[index, column] = item.replace(role, '').strip()

# Calculating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking number of cause_of_death values
print(f'There are {df["cause_of_death"].notna().sum()} values in cause_of_death column.\n')

There are 17342 values in cause_of_death column.

CPU times: total: 3min 24s
Wall time: 3min 24s


<IPython.core.display.Javascript object>

#### Checking Updated `num_categories` Value Counts

In [23]:
# Checking Updated num_categories Value Counts
df["num_categories"].value_counts()

1    88099
2     9704
3      195
0       43
Name: num_categories, dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` and `cause_of_death` for the next iteration.

#### Finding `known_for` Roles and `cause_of_death` in `info_3_0`

In [24]:
# # Obtaining values for column and their counts
# roles_cause_list = df["info_3_0"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [267]:
# # Code to check each value
# roles_cause_list.pop()

<IPython.core.display.Javascript object>

In [262]:
# # Create specific_ro


<IPython.core.display.Javascript object>

In [268]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_cause_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [269]:
# # Example code to quick-check a specific entry
# df[df["info_3_0"] == "United Airlines Flight crash survivor"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category and for `cause_of_death`

In [270]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = [
    "evidence led to ban on sex segregated classified advertising",
    "leading authority on the Constitution of",
    "Acting Prime Minister of the Democratic Republic of the",
    "Prime Minister of the Socialist Federal Republic of",
    "wife of the Prime Minister Konstantinos Mitsotakis",
    "widow of former Prime Minister Sir William McMahon",
    "Principal Private Secretary to the Prime Minister",
    "Press Secretary to Prime Minister Brian Mulroney",
    "last Prime Minister of the People Republic of",
    "son of former Prime Minister Lester B Pearson",
    "brother of former Prime Minister John Major",
    "wife of Prime Minister Alexander Bustamante",
    "Secretary of the Prime Minister Department",
    "Prime Minister of the Federal Republic of",
    "widow of former Prime Minister Takeo Miki",
    "wife of Prime Minister Eddie Fenech Adami",
    "Prime Minister of the Socialist Republic",
    "press secretary for Prime Minister Petre",
    "first wife of Prime Minister Bob Hawke",
    "Chief of Staff to the Prime Minister",
    "Prime Minister of Northern from to",
    "Prime Minister of Czechoslovakia",
    "Spouse of the Prime Minister of",
    "spokesperson for Prime Minister",
    "former Labour Prime Minister of",
    "Prime Minister of Lower Saxony",
    "Prime Minister of the Antilles",
    "Spouse of the Prime Minister",
    "former Prime Minister of the",
    "Prime Minister of Tanganyika",
    "Prime Minister of Macedonia",
    "Prime Minister of Northern",
    "Prime Minister of Bunyoro",
    "Prime Minister of Krajina",
    "former Prime Minister of",
    "Prime Minister of Crimea",
    "Prime Minister of the s",
    "twice Prime Minister of",
    "Prime Minister of South",
    "Prime Minister of Azad",
    "Deputy Prime Minister",
    "Acting Prime Minister",
    "acting Prime Minister",
    "st Prime Minister of",
    "th Prime Minister of",
    "Prime Minister of",
    "argued that social problems contributed to mental illness",
    "former executive director of the Republican National Committee",
    "executive director of the United Nations Population Fund",
    "executive mayor of the George Local Municipality",
    "chief executive of the Medical Research Council",
    "county executive for Prince George County",
    "chief executive of Te Rūnanga o Ngāi Tahu",
    "executive director of the NAACP",
    "executive director of the NHCP",
    "chief executive councillor of",
    "executive director of NFLPA",
    "non profit executive",
    "opposition figure who ran unsuccessfully for President losing to Bakili Muluzi",
    "former President of the Federal Republic",
    "President of the Senate during the Jamaat al Muslimeen coup attempt",
    "President of the International Tribunal for the Law of the Sea",
    "Secretary of Commerce under President Lyndon B Johnson from to",
    "President of the Democratic National Construction Association",
    "President of the Southern Christian Leadership Conference",
    "President of the Regional Council of Languedoc Roussillon",
    "first President of the Grandmothers of the Plaza de Mayo",
    "Assistant Labor Secretary under President Richard Nixon",
    "first President of the Federated States of Micronesia",
    "Vice President of the International Court of Justice",
    "President of Himachal Pradesh Bharatiya Janata Party",
    "mentioned in President Obama election victory speech",
    "President of the Public Establishment of the Palace",
    "President of the Legislative Assembly of Pernambuco",
    "President pro tempore of the North Carolina Senate",
    "President of who murdered thousands of adversaries",
    "President of the Houthi Supreme Political Council",
    "Minister President of the Brussels Capital Region",
    "President of the United Nations General Assembly",
    "President of the United Nations Security Council",
    "President of the National Organization for Women",
    "President of the International Alliance of Women",
    "President of the Union of Democratic Mineworkers",
    "President of the New South Industrial Commission",
    "former Vice President of the People Republic of",
    "President of the New South Legislative Council",
    "President of the Provincial Deputation of Lugo",
    "last President of the People Republic in exile",
    "first female President of the Mescalero Apache",
    "President of University of Student Federation",
    "President of the Federal Constitutional Court",
    "President of the Regional Council of Brittany",
    "President of the Regional Junta of the Azores",
    "as wife of President Frederik Willem de Klerk",
    "President of the Alaska Federation of Natives",
    "President of the Parliament of the Community",
    "member of the Presidential Advisory Council",
    "President pro tempore of the Montana Senate",
    "mother of ex President Luis Alberto Lacalle",
    "Vice President of the National Assembly of",
    "President of the Inter Commission of Women",
    "wife of former President Ibrahim Babangida",
    "President of Akhil Bharatiya Gorkha League",
    "second wife of President Fulgencio Batista",
    "widow of former Vice President Spiro Agnew",
    "President of the South Legislative Council",
    "third party candidate for President of the",
    "Vice President of the Chamber of Deputies",
    "President of the Presidium of the Supreme",
    "President of the Armed Forces of Cabinda",
    "President of the Islamic Salvation Front",
    "President of the Parliament of Cantabria",
    "Vice President of the Court of Appeal of",
    "President of the Executive Council of SR",
    "Minister President of Baden Württemberg",
    "Chief of Staff to President Alpha Condé",
    "Presidential Medal of Freedom recipient",
    "Vice President of the National Assembly",
    "Prohibition Party nominee for President",
    "President of the Bharatiya Janata Party",
    "President of the Women Organization of",
    "President of the National Action Party",
    "President of the Trades Union Congress",
    "President of the Court of Human Rights",
    "President of the Government of Navarre",
    "President of the Political Association",
    "Vice President of the Council of State",
    "President of the Republic of Macedonia",
    "former Counsel under Presidents Carter",
    "first democratically elected President",
    "President of the Congress of Deputies",
    "President of Assembly of the Republic",
    "President of the Constitutional Court",
    "President of the Social Liberal Party",
    "President of the Government of Aragon",
    "President of the Constituent Assembly",
    "Acting President of the Presidency of",
    "President of the Supreme People Court",
    "Vice President of Zhejiang University",
    "President of the Province of Grosseto",
    "President of Broadcasting Corporation",
    "President of the Chamber of Deputies",
    "Principal Secretary to the President",
    "President of the United Steelworkers",
    "President of the Federal Republic of",
    "President of the National Parliament",
    "President of the Province of Naples",
    "President of the University of Guam",
    "Lord President of the Supreme Court",
    "President pro tempore of the Senate",
    "President of Parliamentary Party of",
    "Minister President of the Community",
    "widow of President Salvador Allende",
    "Minister President of Saxony Anhalt",
    "President of University of Southern",
    "President of the National Assembly",
    "President of the National Congress",
    "President of the Province of Udine",
    "Secretary General of the President",
    "President of the courts of appeals",
    "President of the Regional Council",
    "President of the Council of State",
    "President of Province of Alicante",
    "President of the National Council",
    "President of Bharatiya Jana Sangh",
    "President of the Examination Yuan",
    "advisor to President Jimmy Carter",
    "first President of the Federation",
    "President of the Legislative Yuan",
    "President of the Liberal Party of",
    "President of the Mescalero Apache",
    "President of the Seneca Nation of",
    "brother of President Hamid Karzai",
    "th President of the Presidency of",
    "Executive Vice President of Yukos",
    "President of the Xunta of Galicia",
    "spokesman of the President Office",
    "President of the Illinois Senate",
    "President of the Bar Association",
    "President of Deutsche Bundesbank",
    "President of the Udmurt Republic",
    "President of the Family Division",
    "President of the Supreme Council",
    "separatist President of Abkhazia",
    "President of the Court of Appeal",
    "President of the Central Bank of",
    "President of Chamber of Deputies",
    "President of the Pan Parliament",
    "candidate for President of a in",
    "first President of Azad Kashmir",
    "President of the People Chamber",
    "two time Presidential candidate",
    "President of the Navajo Nation",
    "President of National Assembly",
    "President of the Liberal Party",
    "President of the Free Alliance",
    "President of the Federation of",
    "President of the Supreme Court",
    "President of PEN International",
    "President of the Confederation",
    "President of the Victorian RSL",
    "President of Valencian Courts",
    "President of National Council",
    "President of the People Party",
    "President of the Maine Senate",
    "President of the State Senate",
    "President of Jamaat e Islami",
    "President of the Republic of",
    "President of the Cook County",
    "candidate for Vice President",
    "advisor to President Kennedy",
    "President of Bar Association",
    "President of the Parliament",
    "President of the New Senate",
    "President of the Commission",
    "President of Bophuthatswana",
    "President of the Royal Mint",
    "Borough President of Queens",
    "President of the Bundesbank",
    "fourth President of from to",
    "President of Pan Parliament",
    "President of Czechoslovakia",
    "President of the Bundestag",
    "President of the Authority",
    "Vice President of the USSR",
    "President of Oromia Region",
    "President of the Senate of",
    "Vice President of FRELIMO",
    "former President of North",
    "President of the Republic",
    "th Vice President of the",
    "eighth Vice President of",
    "President of Upper Volta",
    "President of the Senate",
    "President of Parliament",
    "President of Polynesia",
    "first female President",
    "President of y from to",
    "Second Vice President",
    "President of Zanzibar",
    "President of Puntland",
    "President of Hadassah",
    "former President of y",
    "de facto President of",
    "President of Chechnya",
    "President of Sardinia",
    "President of Madeira",
    "President of Krajina",
    "President of Galicia",
    "Third Vice President",
    "President of MCA Inc",
    "President of the NPU",
    "first President of a",
    "Vice State President",
    "first Vice President",
    "Federal President of",
    "President of the ZOA",
    "President of the NRC",
    "President of the PCA",
    "President of Sicily",
    "President of West y",
    "Temporary President",
    "former President of",
    "President of North",
    "President of SCBAP",
    "President of Kasaï",
    "President of since",
    "President of South",
    "President of USDAW",
    "Vice President of",
    "rd President of a",
    "st President of s",
    "Interim President",
    "Acting President",
    "acting President",
    "Senate President",
    "President of RAI",
    "nd President of",
    "Arunachal Pradesh MLA for Changlang North",
    "Columbia MLA for Vancouver Point Grey",
    "Prince Edward Island MLA for rd Kings",
    "Andhra Pradesh MLA for Chintalapudi",
    "MLA for Rockwood Iberville Lakeside",
    "Alberta MLA for Edmonton Mill Woods",
    "MLA for Dalhousie Restigouche East",
    "MLA from the Northwest Territories",
    "MLA for Vancouver Little Mountain",
    "Andhra Pradesh MLA for Allagadda",
    "Madhya Pradesh MLA for Raghogarh",
    "Alberta MLA for Calgary Glenmore",
    "Alberta MLA for Lethbridge West",
    "Madhya Pradesh MLA for Kasrawad",
    "Victorian MLA for Ballarat West",
    "Queensland MLA for Ipswich West",
    "Alberta MLA for Calgary Currie",
    "Columbia MLA for Prince Rupert",
    "Queensland MLA for Port Curtis",
    "Yukon MLA for McIntyre Takhini",
    "MLA for Edmonton Sherwood Park",
    "Nova Scotia MLA for West Kings",
    "Uttar Pradesh MLA for Deoband",
    "Manitoba MLA for Rupertsland",
    "Punjab MLA for Amritsar West",
    "Chhattisgarh MLA for Kharsia",
    "Uttar Pradesh MLA for Bilari",
    "MLA for Boundary Similkameen",
    "Victoria MLA for Mooroolbark",
    "Odisha MLA for Kissan Nagar",
    "MLA for Saskatoon Northwest",
    "West Bengal MLA for Amdanga",
    "Columbia MLA for Mackenzie",
    "Alberta MLA for Clover Bar",
    "Western MLA for Swan Hills",
    "MLA for Fundy River Valley",
    "Northwest Territories MLA",
    "Manitoba MLA for Kildonan",
    "MLA for Tuensang Sadar II",
    "MLA for Whitehorse Centre",
    "MLA for South Peace River",
    "Nadu MLA for Kadayanallur",
    "Prince Edward Island MLA",
    "Nadu MLA for Tiruchendur",
    "MLA for East Londonderry",
    "Columbia MLA for Saanich",
    "four time Karnataka MLA",
    "Kerala MLA for Kuttanad",
    "Queensland MLA for Cook",
    "New South MLA for Bulli",
    "MLA for Dartmouth South",
    "Northern Territory MLA",
    "MLA for Calgary Currie",
    "MLA for Rossland Trail",
    "MLA for Nellore Rural",
    "Arunachal Pradesh MLA",
    "Sindh MLA for Karachi",
    "MLA for Okanagan West",
    "Sarawak MLA for Krian",
    "Himachal Pradesh MLA",
    "MLA for Antrim North",
    "Gilgit Baltistan MLA",
    "MLA for Pandavapura",
    "MLA for Cape Centre",
    "six time Odisha MLA",
    "MLA for Alipurduars",
    "Nova Scotia MLA for",
    "MLA for Calgary Bow",
    "Andhra Pradesh MLA",
    "Madhya Pradesh MLA",
    "Rio de Janeiro MLA",
    "Santa Catarina MLA",
    "MLA for Surat East",
    "MLA for Memramcook",
    "MLA for Upper Bann",
    "MLA for North Down",
    "Uttar Pradesh MLA",
    "New Brunswick MLA",
    "Malacca State MLA",
    "MLA for Nathdwara",
    "NSW MLA for Dubbo",
    "MLA for Inverness",
    "MLA for St Albert",
    "MLA for Fermanagh",
    "Saskatchewan MLA",
    "Chhattisgarh MLA",
    "MLA for Waverley",
    "Minas Gerais MLA",
    "Maharashtra MLA",
    "West Bengal MLA",
    "Nova Scotia MLA",
    "Uttarakhand MLA",
    "MLA for Noorpur",
    "Mato Grosso MLA",
    "Perak State MLA",
    "MLA of Columbia",
    "MLA for Burrows",
    "MLA for Palghar",
    "MLA for Caracas",
    "Queensland MLA",
    "Puducherry MLA",
    "MLA of Alberta",
    "MLA for Altona",
    "Karnataka MLA",
    "Victorian MLA",
    "Rajasthan MLA",
    "São Paulo MLA",
    "New South MLA",
    "Meghalaya MLA",
    "Telangana MLA",
    "Jharkhand MLA",
    "Columbia MLA",
    "Manitoba MLA",
    "Selangor MLA",
    "Maranhão MLA",
    "Nagaland MLA",
    "Kelantan MLA",
    "Alberta MLA",
    "Haryana MLA",
    "Gujarat MLA",
    "Western MLA",
    "Paraíba MLA",
    "Sarawak MLA",
    "Kerala MLA",
    "Odisha MLA",
    "Paraná MLA",
    "Punjab MLA",
    "Bihar MLA",
    "Delhi MLA",
    "Assam MLA",
    "Yukon MLA",
    "Piauí MLA",
    "Johor MLA",
    "Perak MLA",
    "Goiás MLA",
    "Nadu MLA",
    "Goa MLA",
    "ACT MLA",
    "WA MLA",
    "AP MLA",
    "four time MLA",
    "MLA",
    "State Senator for Connecticut th District",
    "former Democratic Senator from Minnesota",
    "former Republican Senator from Maryland",
    "former Republican Senator for Nevada",
    "former North Carolina state Senator",
    "Senator of the College of Justice",
    "Democratic Senator from Wisconsin",
    "Senator from Illinois from to",
    "Senator for Dublin University",
    "North Carolina State Senator",
    "Senator for Loire Atlantique",
    "Senator for Alpes Maritimes",
    "Senator from Massachusetts",
    "former Labor Party Senator",
    "Senator from New Hampshire",
    "Senator for Borno Central",
    "Senator from Haute Savoie",
    "former Democratic Senator",
    "Senator for New Hampshire",
    "Washington State Senator",
    "Senator for Kaduna South",
    "Senator for Kaduna State",
    "Tennessee State Senator",
    "Minnesota State Senator",
    "Republican Senator from",
    "Senator for Nova Scotia",
    "Wisconsin State Senator",
    "Senator from Tennessee",
    "Senator for Queensland",
    "Nebraska State Senator",
    "Oklahoma State Senator",
    "served as Senator from",
    "Senator for Abia North",
    "Missouri State Senator",
    "Maryland State Senator",
    "Michigan State Senator",
    "Senator for New South",
    "Senator from Illinois",
    "Arizona State Senator",
    "Indiana State Senator",
    "New Hampshire Senator",
    "Senator from Maryland",
    "Vermont State Senator",
    "Senator from Alabama",
    "Senator for Victoria",
    "Senator from Montana",
    "Senator for Manitoba",
    "Senator for Rondônia",
    "Senator from Wyoming",
    "Oregon State Senator",
    "Nevada State Senator",
    "Senator for Western",
    "Senator from Kansas",
    "Senator from Hawaii",
    "Senator from Alaska",
    "Utah State Senator",
    "Iowa State Senator",
    "Senator from Ohio",
    "Senator from Utah",
    "Senator for Osun",
    "former Senator",
    "State Senator",
    "Senator from",
    "Senator for",
    "New Senator",
    "Ambassador to the United Nations Commission on the Status of Women",
    "Ambassador of the Arab League to the United Nations",
    "Ambassador to the United Nations for",
    "Ambassador to the United Nations",
    "editor in chief of Ambassador to",
    "served as Ambassador to from to",
    "Ambassador to the Holy See",
    "Ambassador to the Republic",
    "Ambassador to Upper Volta",
    "Ambassador to the UN and",
    "Ambassador to the USSR",
    "Ambassador to the and",
    "former Ambassador to",
    "Deputy Ambassador to",
    "Ambassador to West y",
    "Ambassador to East y",
    "Ambassador to Arabia",
    "Ambassador of to the",
    "Ambassador to istan",
    "Ambassador to Union",
    "Ambassador at large",
    "Ambassador to the",
    "Ambassador to and",
    "Ambassador to y",
    "Ambassador to s",
    "Ambassador to",
    "known for playing a major role in Secretary of State Henry Kissinger shuttle diplomacy",
    "known for Munsinger Affair",
    "Director of the Office of Nuclear Reactor Regulation",
    "segregationist politician",
    "Forza Europa politician",
    "barrister politician",
    "and politician",
    "politician",
]
politics_govt_law = sorted(
    list(set(politics_govt_law)), key=lambda x: len(x), reverse=True
)

arts = [
    "writer",
    "executive director of the Philharmonic",
    "executive producer of the soap opera",
    "chief executive of VF Corporation",
    "film & TV actor; Motown executive",
    "NFLPA executive director",
    "studio executive",
    "executive at CNN",
    "media executive",
    "influential in language poetry movement",
    'called the "People poet of Dagestan"',
    "poet laureate of Iowa",
    "Iowa poet laureate",
    "poet",
    "known as a leading journalist opposed to",
    "first woman to work as a photojournalist",
    "adventure photojournalist",
    "and newspaper journalist",
    "television journalist",
    "cultural journalist",
    "journalist artist",
    "music journalist",
    "radio journalist",
    "photojournalist",
    "and journalist",
    "by singer Bertrand Cantat",
    "singer in the Buena Vista Social Club",
    "ex husband of singer Tina Turner",
    "backup singer for Elvis Presley",
    "mother of singer Stevie Wonder",
    "playback singer",
    "country singer",
    "radio singer",
    "opera singer",
    "cuplé singer",
    "folk singer",
    "and singer",
    "folksinger",
    "widow of composer Sir William Walton",
    "composer of the national anthem",
    "and film composer",
    "music composer",
    "composer",
    "benefactor of the Mondavi Center",
    "first black actor to appear on",
    "Academy Award nominated actor",
    "son of actors Margo",
    "theatre actor",
    "voice actor",
    "comic actor",
    "film actor",
    "and actor",
    "playwright",
]
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)

sports = [
    "son of Prime Minister Pierre Trudeau",
    "executive chairman of the Asian Tour",
    "executive director of the MLBPA",
    "minor league baseball executive",
    "President of the Green Bay Packers",
    "President of FIBA",
    "owner of the Ottawa Senators",
]
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True)

sciences = [
    "worked on leukemia treatments",
    "first chief executive of Amgen",
    "health supplement executive",
    "natural resource executive",
]
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True)

business_farming = [
    "chief executive of the Shipowners' Association",
    "executive director of the Business Roundtable",
    "chief executive of the Eastman Kodak Company",
    "and the chief executive of Aerospace",
    "manufacturing executive",
    "executive with Converse",
    "construction executive",
    "chief executive of IBM",
    "advertising executive",
    "financial executive",
    "business executive",
    "airline executive",
    "casino executive",
    "steel executive",
    "auto executive",
    "oil executive",
    "President of Amtrak",
    "founder of Tanger Factory Outlet Centers",
]
business_farming = sorted(
    list(set(business_farming)), key=lambda x: len(x), reverse=True
)

academia_humanities = [
    "leading authority on the art of medieval",
    "a leading authority on the Union and",
    "authority on secret societies",
    "authority on Native culture",
    "authority on literature",
    "authority on art",
    "executive dean of Nelson Mandela University",
    "executive director of",
    "President of McMaster University",
    "President of Wesleyan University",
    "President of Longwood University",
    "President of Columbia University",
    "President of Lingnan University",
    "President of Peking University",
    "President of Tehran University",
]
academia_humanities = sorted(
    list(set(academia_humanities)), key=lambda x: len(x), reverse=True
)

law_enf_military_operator = [
    "deputy political commissar of Shenyang Military Region",
    "deputy inspector general of the National Police",
    "deputy commander of RAF Strike Command",
    "deputy commander of the PLA Air Force",
    "deputy chief of staff of the Army",
    "deputy director of Shin Bet",
    "deputy chief of staff",
]
law_enf_military_operator = sorted(
    list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True
)

spiritual = [
    "first Native to become a general authority of The Church of Jesus Christ of Latter day Saints",
    "general authority of The Church of Jesus Christ of Latter day Saints",
    "general authority of the LDS Church",
    "chief executive of Concern Worldwide",
]
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)

social = [
    "first executive director of Gay Men Health Crisis",
]
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)

crime = [
    "killer of dissident journalist Henry Liu",
]
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)

event_record_other = [
    "after killing his old autistic son",
    "of standing trial for war crimes",
    "whose accident photos were released onto internet",
    "world record holder for longest fall",
    "USAID contractor kidnapped by al Qaeda",
    "United Airlines Flight crash survivor",
]
event_record_other = sorted(
    list(set(event_record_other)), key=lambda x: len(x), reverse=True
)

other_species = [
    "co owned by Prime Minister Jacinda Ardern",
]
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True)

cause_of_death = [
    "suicide",
    "chronic lymphocytic leukemia",
    "complications from leukemia",
    "chronic lymphomic leukemia",
    "acute myelogenous leukemia",
    "complications of leukemia",
    "acute myeloid leukemia",
    "lymphoblastic leukemia",
    "acute myeloma leukemia",
    "lymphocytic leukemia",
    "plasma cell leukemia",
    "myelogenous leukemia",
    "lymphoid leukemia",
    "myeloid leukemia",
    "chronic leukemia",
    "acute leukemia",
    "complications from respiratory illness",
    "complications from a prolonged illness",
    "complications of respiratory illness",
    "infection from a chronic illness",
    "respiratory related illness",
    "following a long illness",
    "after prolonged illness",
    "after a lengthy illness",
    "kidney related illness",
    "after a short illness",
    "age related illnesses",
    "liver related illness",
    "after a brief illness",
    "after serious illness",
    "AIDS related illness",
    "after a long illness",
    "drug related illness",
    "degenerative illness",
    "after short illness",
    "respiratory illness",
    "after brief illness",
    "HIV related illness",
    "after long illness",
    "pulmonary illness",
    "prolonged illness",
    "lingering illness",
    "after an illness",
    "stomach illness",
    "kidney illness",
    "short illness",
    "heart illness",
    "liver illness",
    "brief illness",
    "after illness",
    "long illness",
    "lung illness",
    "malignant hypertension leading to kidney failure",
    "pulmonary failure as a complication of diabetes",
    "massive organ failure after being imprisoned",
    "respiratory failure due to diverticulitis",
    "dementia with complicating kidney failure",
    "liver failure as a result of hepatitis C",
    "complications from respiratory failure",
    "kidney failure after wrestling injury",
    "liver failure from substance abuse",
    "complications from kidney failure",
    "organ failure due to dengue fever",
    "liver failure after hunger strike",
    "complications from renal failure",
    "complications from liver failure",
    "euthanized after kidney failure",
    "complications of kidney failure",
    "complications of organ failure",
    "chronic respiratory failure",
    "hypoxic respiratory failure",
    "power failure to iron lung",
    "cardio respiratory failure",
    "cardiorespiratory failure",
    "acute respiratory failure",
    "left ventricular failure",
    "suspected kidney failure",
    "cardiopulmonary failure",
    "multiple organ failure",
    "respiratory failure",
    "bone marrow failure",
    "acute renal failure",
    "pulmonary failure",
    "failure to thrive",
    "bronchial failure",
    "kidney failure",
    "renal failure",
    "liver failure",
    "organ failure",
    "lung failure",
    "complications from injuries sustained in traffic accident",
    "from injuries sustained in a motorcycle accident",
    "head injuries resulting from domestic accident",
    "fatal neck injury sustained in an accident in",
    "injuries sustained in campdrafting accident",
    "injuries sustained in a motorcycle accident",
    "complications following a traffic accident",
    "injuries sustained in skydiving accident",
    "killed in an accident during a tire test",
    "injuries sustained in a jetboat accident",
    "complications following skiing accident",
    "diffuse axonal injury from car accident",
    "complications after a bicycle accident",
    "injuries from a motor scooter accident",
    "injuries sustained in bicycle accident",
    "accidental prescription drug overdose",
    "accidental combined drug intoxication",
    "injuries sustained in skiing accident",
    "injuries sustained in a car accident",
    "injuries sustained from car accident",
    "accidental overdose of pain killers",
    "accidental overdose of hydrocodone",
    "complications from skiing accident",
    "consequences of a vehicle accident",
    "accidental overdose of barbiturate",
    "injuries sustained in an accident",
    "accidental hydromorphone overdose",
    "complications from a car accident",
    "apparent accidental drug overdose",
    "head injury from bicycle accident",
    "complications after car accident",
    "complications from ATV accident",
    "accidental overdose of alcohol",
    "injuries sustained in accident",
    "accidental medication overdose",
    "injuries from traffic accident",
    "complications of car accident",
    "injuries from accidental fall",
    "accidental alcohol poisoning",
    "accidental fentanyl overdose",
    "accidental cocaine overdose",
    "accidental heroin overdose",
    "motorcycle stunt accident",
    "Formula Two race accident",
    "accidental drug overdose",
    "ski accident in the Alps",
    "accident during practice",
    "hot air balloon accident",
    "mountaineering accident",
    "accidental asphyxiation",
    "rock climbing accident",
    "swimming pool accident",
    "accidental head trauma",
    "conveyor belt accident",
    "drunk driving accident",
    "accidental suffocation",
    "BASE jumping accident",
    "scuba diving accident",
    "hang gliding accident",
    "falling accident on K",
    "kiteboarding accident",
    "team chasing accident",
    "accidental impalement",
    "road traffic accident",
    "work related accident",
    "base jumping accident",
    "free diving accident",
    "paragliding accident",
    "agriculture accident",
    "motorcycle accident",
    "automobile accident",
    "accidental overdose",
    "accidental drowning",
    "industrial accident",
    "accidental fentanyl",
    "helicopter accident",
    "snowmobile accident",
    "gyrocopter accident",
    "single car accident",
    "automotive accident",
    "skydiving accident",
    "accidental choking",
    "workplace accident",
    "submarine accident",
    "vehicular accident",
    "wrestling accident",
    "rally car accident",
    "climbing accident",
    "domestic accident",
    "aviation accident",
    "drowning accident",
    "swimming accident",
    "railroad accident",
    "air show accident",
    "traffic accident",
    "boating accident",
    "bicycle accident",
    "cycling accident",
    "farming accident",
    "hunting accident",
    "tractor accident",
    "jet ski accident",
    "fishing accident",
    "paddock accident",
    "choking accident",
    "cricket accident",
    "surfing accident",
    "gliding accident",
    "jumping accident",
    "scooter accident",
    "racing accident",
    "accidental fall",
    "skiing accident",
    "hiking accident",
    "diving accident",
    "marine accident",
    "accidental drug",
    "motor accident",
    "crane accident",
    "road accident",
    "race accident",
    "auto accident",
    "fire accident",
    "car accident",
    "ATV accident",
    "bus accident",
    "gun accident",
    "murdered while attempting to arrest an Islamic terrorist",
    "murdered during the Melbourne gangland killings",
    "murdered by the United Self Defense Forces of",
    "murdered by the Loyalist Volunteer Force",
    "murdered by serial killer Andrew Cunanan",
    "murdered during a voyage on a catamaran",
    "murdered while working at McDonald",
    "murdered along with his family",
    "murdered by a Hutu rebel group",
    "murdered by organized crime",
    "murdered in line of duty",
    "murdered during burglary",
    "murdered during robbery",
    "murdered in a shooting",
    "murdered in Jasenovac",
    "murdered around May",
    "murdered in prison",
    "murdered in Anapu",
    "murdered by a mob",
    "murdered abroad",
    "complications following neck injuries from a fall",
    "complications of a brain injury after a fall",
    "complications from surgery following a fall",
    "blunt force trauma after fall from vehicle",
    "complications of a head injury from fall",
    "brain injury from fall during beating",
    "complications of injuries from a fall",
    "brain injury from competition fall",
    "injuries sustained in racing fall",
    "surgical complications after fall",
    "cerebral hemorrhage after falling",
    "head injuries sustained in a fall",
    "fall when climbing Mount Everest",
    "cerebral hemorrhage from a fall",
    "head trauma from treadmill fall",
    "complications following a fall",
    "injuries sustained from a fall",
    "head injuries following a fall",
    "cerebral hemorrhage after fall",
    "fall from a piece of equipment",
    "neck injury from bicycle fall",
    "injuries from accidental fall",
    "injuries sustained in a fall",
    "injuries sustained from fall",
    "brain hemorrhage after fall",
    "fall during Paris Nice race",
    "complications after a fall",
    "fall from apartment window",
    "euthanized after race fall",
    "euthanised after race fall",
    "complications from a fall",
    "head injuries from a fall",
    "complications after fall",
    "broken neck after a fall",
    "crushed by falling horse",
    "brain injury from a fall",
    "complications of a fall",
    "complications from fall",
    "head injuries from fall",
    "struck by falling tree",
    "head injury from fall",
    "complications of fall",
    "falling accident on K",
    "injuries from a fall",
    "fall from a building",
    "trauma due to a fall",
    "fall on Nanga Parbat",
    "hit by falling tree",
    "fall while training",
    "fall from a bicycle",
    "fall from building",
    "injuries from fall",
    "fall from mountain",
    "injury from a fall",
    "fall from balcony",
    "rockclimbing fall",
    "fall from a cliff",
    "fall from window",
    "accidental fall",
    "fall from horse",
    "fall from cliff",
    "fall from roof",
    "fall from boat",
    "climbing fall",
    "falling tree",
    "fall at home",
    "rock fall",
    "lung infection while battling a bone lymphoma",
    "lymphoma due to chronic active EBV infection",
    "complications of non Hodgkin lymphoma",
    "complications from B cell lymphoma",
    "central nervous system lymphoma",
    "non Hodgkin follicular lymphoma",
    "chronic lymphocytic lymphoma",
    "T cell lymfoblastic lymphoma",
    "complications from lymphoma",
    "from lymphoma complications",
    "complications of lymphoma",
    "cutaneous B cell lymphoma",
    "T lymphoblastic lymphoma",
    "lymphoma of brain cells",
    "non Hodgkin’s lymphoma",
    "AIDS induced lymphoma",
    "non Hodgkin lymphoma",
    "mantle cell lymphoma",
    "Non hodgkin lymphoma",
    "Non Hodgkin lymphoma",
    "malignant lymphoma",
    "cerebral lymphoma",
    "Hodgkin lymphoma",
    "Burkitt lymphoma",
    "T cell lymphoma",
    "B cell lymphoma",
    "lymphoma",
    "stabbed in the chest by a stingray barb",
    "stabbed during Westminster attack",
    "stabbed to death by his son",
    "stabbed to death",
    "stabbed",
    "injuries sustained in the Lokomotiv Yaroslavl plane crash",
    "crash during race practice for the Superbike Championship",
    "drowned in helicopter crash while filming",
    "head injuries sustained in a car crash",
    "victim of the Martinsville plane crash",
    "injuries sustained in a plane crash",
    "injuries sustained in a car crash",
    "after crashing at Suzuka on April",
    "injuries sustained in race crash",
    "bicycle crash during the Tour de",
    "head injuries from a race crash",
    "plane crash of Swissair Flight",
    "brain injuries from race crash",
    "survivor of LaMia Flight crash",
    "victim of the City plane crash",
    "head injury from car crash",
    "injuries from a race crash",
    "crash during Daytona race",
    "injuries from a car crash",
    "car crash in Menlo Park",
    "Farnborough air crash",
    "crash during practice",
    "survivor of crash",
    "paragliding crash",
    "competition crash",
    "helicopter crash",
    "motorcycle crash",
    "spaceplane crash",
    "snowmobile crash",
    "automobile crash",
    "racetrack crash",
    "airplane crash",
    "wingsuit crash",
    "training crash",
    "aircraft crash",
    "bicycle crash",
    "airshow crash",
    "jet car crash",
    "glider crash",
    "rocket crash",
    "racing crash",
    "train crash",
    "rally crash",
    "race crash",
    "car crash",
    "air crash",
    "bus crash",
    "aircrash",
]
cause_of_death = sorted(list(set(cause_of_death)), key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [271]:
# Hard-coding cause_of_death for entry for clarity of value
index = df[df["link"] == "https://en.wikipedia.org/wiki/Nikki_Catsouras"].index
df.loc[index, "cause_of_death"] = "car accident"

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [272]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting `known_for` Categories and `cause_of_death` Values from `info_3_0`

In [273]:
%%time

# Column to check
column = 'info_3_0'

# Start dataframe
dataframe = df[df[column].notna()]

# For loop to find cause in column and extract it to cause_of_death
for cause in cause_of_death:
    for index in dataframe.index:
        item = df.loc[index, column]
        if item:
            if cause in item:
                df.loc[index, 'cause_of_death'] = cause
                df.loc[index, column] = item.replace(cause, '').strip()
                
# For loop to find role in column and extract it as category
for category, category_lst in known_for_dict.items():
    for role in category_lst:
        for index in dataframe.index:
                item = df.loc[index, column]
                if item:
                    if role in item:
                        df.loc[index, category] = 1
                        df.loc[index, column] = item.replace(role, '').strip()

# Calculating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking number of cause_of_death values
print(f'There are {df["cause_of_death"].notna().sum()} values in cause_of_death column.\n')

There are 19841 values in cause_of_death column.

CPU times: total: 5min 19s
Wall time: 5min 19s


<IPython.core.display.Javascript object>

#### Checking Updated `num_categories` Value Counts

In [274]:
# Checking Updated num_categories Value Counts
df["num_categories"].value_counts()

1    87836
2     9956
3      207
0       42
Name: num_categories, dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` and `cause_of_death` for the next iteration.

#### Finding `known_for` Roles and `cause_of_death` in `info_3_0`

In [ ]:
# Obtaining values for column and their counts
roles_cause_list = df["info_3_0"].value_counts(ascending=True).index.tolist()

In [ ]:
# Code to check each value
roles_cause_list.pop()

In [ ]:
# # Create specific_roles_cause_list for above popped value
# specific_roles_cause_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_3_0"].notna()].index
#             if "general" in df.loc[index, "info_3_0"]
#         ],
#         "info_3_0",
#     ]
#     .value_counts()
#     .index.tolist()
# )

In [ ]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_cause_list, key=lambda x: len(x), reverse=True)

In [ ]:
# # Example code to quick-check a specific entry
# df[df["info_3_0"] == "outlaw country music singer songwriter"]

#### Creating Lists for Each `known_for` Category and for `cause_of_death`

In [ ]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = []
politics_govt_law = sorted(list(set(politics_govt_law)), key=lambda x: len(x), reverse=True)  

arts = []
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)  

sports = []
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True) 

sciences = []
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True) 

business_farming = []
business_farming = sorted(list(set(business_farming)), key=lambda x: len(x), reverse=True)  

academia_humanities = []
academia_humanities = sorted(list(set(academia_humanities)), key=lambda x: len(x), reverse=True)  

law_enf_military_operator = []
law_enf_military_operator = sorted(list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True)  

spiritual = []
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)  

social = []
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)  

crime = []
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)  

event_record_other = []
event_record_other = sorted(list(set(event_record_other)), key=lambda x: len(x), reverse=True)  

other_species = []
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True) 

cause_of_death = []
cause_of_death = sorted(list(set(cause_of_death)), key=lambda x: len(x), reverse=True)

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [ ]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

#### Extracting `known_for` Categories and `cause_of_death` Values from `info_3_0`

In [ ]:
%%time

# Column to check
column = 'info_3_0'

# Start dataframe
dataframe = df[df[column].notna()]

# For loop to find cause in column and extract it to cause_of_death
for cause in cause_of_death:
    for index in dataframe.index:
        item = df.loc[index, column]
        if item:
            if cause in item:
                df.loc[index, 'cause_of_death'] = cause
                df.loc[index, column] = item.replace(cause, '').strip()
                
# For loop to find role in column and extract it as category
for category, category_lst in known_for_dict.items():
    for role in category_lst:
        for index in dataframe.index:
                item = df.loc[index, column]
                if item:
                    if role in item:
                        df.loc[index, category] = 1
                        df.loc[index, column] = item.replace(role, '').strip()

# Calculating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking number of cause_of_death values
print(f'There are {df["cause_of_death"].notna().sum()} values in cause_of_death column.\n')

#### Checking Updated `num_categories` Value Counts

In [ ]:
# Checking Updated num_categories Value Counts
df["num_categories"].value_counts()

#### Observations:
- We will proceed to rebuild `known_for_dict` and `cause_of_death` for the next iteration.

In [275]:
print("dunzo!")

# Sound notification when cell executes
chime.success()

dunzo!


<IPython.core.display.Javascript object>

#### Finding `known_for` Roles and `cause_of_death` in `info_3_0`

In [ ]:
# Obtaining values for column and their counts
roles_cause_list = df["info_3_0"].value_counts(ascending=True).index.tolist()

In [ ]:
# Code to check each value
roles_cause_list.pop()

In [ ]:
# # Create specific_roles_cause_list for above popped value
# specific_roles_cause_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_3_0"].notna()].index
#             if "general" in df.loc[index, "info_3_0"]
#         ],
#         "info_3_0",
#     ]
#     .value_counts()
#     .index.tolist()
# )

In [ ]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_cause_list, key=lambda x: len(x), reverse=True)

In [ ]:
# # Example code to quick-check a specific entry
# df[df["info_3_0"] == "outlaw country music singer songwriter"]

#### Creating Lists for Each `known_for` Category and for `cause_of_death`

In [ ]:
# Creating lists for each category and sorting by decreasing length and removing duplicates

politics_govt_law = []
politics_govt_law = sorted(list(set(politics_govt_law)), key=lambda x: len(x), reverse=True)  

arts = []
arts = sorted(list(set(arts)), key=lambda x: len(x), reverse=True)  

sports = []
sports = sorted(list(set(sports)), key=lambda x: len(x), reverse=True) 

sciences = []
sciences = sorted(list(set(sciences)), key=lambda x: len(x), reverse=True) 

business_farming = []
business_farming = sorted(list(set(business_farming)), key=lambda x: len(x), reverse=True)  

academia_humanities = []
academia_humanities = sorted(list(set(academia_humanities)), key=lambda x: len(x), reverse=True)  

law_enf_military_operator = []
law_enf_military_operator = sorted(list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True)  

spiritual = []
spiritual = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)  

social = []
social = sorted(list(set(social)), key=lambda x: len(x), reverse=True)  

crime = []
crime = sorted(list(set(crime)), key=lambda x: len(x), reverse=True)  

event_record_other = []
event_record_other = sorted(list(set(event_record_other)), key=lambda x: len(x), reverse=True)  

other_species = []
other_species = sorted(list(set(other_species)), key=lambda x: len(x), reverse=True) 

cause_of_death = []
cause_of_death = sorted(list(set(cause_of_death)), key=lambda x: len(x), reverse=True)

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [ ]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

#### Extracting `known_for` Categories and `cause_of_death` Values from `info_3_0`

In [ ]:
%%time

# Column to check
column = 'info_3_0'

# Start dataframe
dataframe = df[df[column].notna()]

# For loop to find cause in column and extract it to cause_of_death
for cause in cause_of_death:
    for index in dataframe.index:
        item = df.loc[index, column]
        if item:
            if cause in item:
                df.loc[index, 'cause_of_death'] = cause
                df.loc[index, column] = item.replace(cause, '').strip()
                
# For loop to find role in column and extract it as category
for category, category_lst in known_for_dict.items():
    for role in category_lst:
        for index in dataframe.index:
                item = df.loc[index, column]
                if item:
                    if role in item:
                        df.loc[index, category] = 1
                        df.loc[index, column] = item.replace(role, '').strip()

# Calculating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking number of cause_of_death values
print(f'There are {df["cause_of_death"].notna().sum()} values in cause_of_death column.\n')

#### Checking Updated `num_categories` Value Counts

In [ ]:
# Checking Updated num_categories Value Counts
df["num_categories"].value_counts()

#### Observations:
- We will proceed to rebuild `known_for_dict` and `cause_of_death` for the next iteration.

#### Observations:
- It is time to export our dataframe and start a new notebook.

### Exporting Dataset to SQLite Database [wp_life_expect_clean6.db]()

In [ ]:
# # Exporting dataframe

# # Saving dataset in a SQLite database
# conn = sql.connect("wp_life_expect_clean6.db")
# df.to_sql("wp_life_expect_clean", conn, index=False)

# # Chime notification when cell executes
# chime.success()

# [Proceed to Data Cleaning Part ]()